# 2: Align DNA and RNA FISH images

1) Get global coorrdinates for DNA and RNA FISH beads.
2) Get local and global alignment for images.
3) Apply alignment to images.

For more details about required in and output please look at the individual notebooks used in the pipeline.

In [ ]:
import os
import queue
import pandas as pd
from pathlib import Path
from datetime import datetime
import papermill as pm
from multiprocessing import Pool
import concurrent.futures

def run_notebook(parameters,notebook_to_run,parameters_common={},out= '/dev/null'):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
            notebook_to_run,
            out,
            parameters=parameters)

# 1) Get global coordinates

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"},
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

### common parameters
parameters_common = {
    "raw_subpath" : "raw", # subpath where raw h5/msr images are saved
    "spots_subpath": "detections_beads/merge.csv", # single or multiple (*) spot files
    "out_subpath" : "detections_beads/", #w here to save results
    
    "coordinate_column_names" : ['z', 'y', 'x'], # naming of xyz coordinates
    "global_coordinate_column_names" : ['z_global_um', 'y_global_um', 'x_global_um'], # how to call global coordinate columns
    "image_file_column_name" : 'img', # name of img file column
    
    "use_h5_for_metadata" : False, # use h5 (faster) or msr? 
}

notebook_to_run = "/home/stumberger/ep2024/subscripts/get_global_coordinates_sted.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 2) Get alignment for images
Plase always chcek that the alignment is correct, by going to the original `ep2024/subscripts/find_transformations_sted.ipynb` script and checking the images in step 3).

In [ ]:
parameters_list = [
    {
        "detection_table1_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/detections_beads/merge_global_coords.csv",
        "detection_table2_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/detections_beads/merge_global_coords.csv"
    }
]

parameters_common = {
    "json_save_path_g": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/transformations.json",
    "json_save_path_l":"/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/transformations_local_1round.json",
    "coordinate_columns_yx" : ['x_global_um', 'y_global_um'],
    "coordinate_column_z": 'z_global_um',
    "image_file_column" : "img",
    "summary_df_save_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/alignment_accuracy.csv"
}
    
notebook_to_run = "/home/stumberger/ep2024/subscripts/find_transformations_sted.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 3) Align bead images

In [ ]:
parameters_list = [
    {
        "base_path_target": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/",
        "base_path_moving": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"
    }
]

parameters_common = {
    "alignment_params_file" : '/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/transformations_local_1round.json',
    "exclude_pattern_target" : "sted",
    "include_pattern_target" : None,
    "exclude_pattern_moving" : None,
    "include_pattern_moving" : None,

    "channels_to_include_target" : [2, ],
    "channels_to_include_moving" : [1, ],
    
    # whether to fuse multiple moving images
    # if False, will only transform the one with the highest overlap, ignoring other moving tiles at the border of target image
    "fuse_multiple_moving" : True,
    
    # "out_subdir" : 'aligned_beads_correlation',
    "out_subdir" : 'aligned_beads',
    
    # whether to save projections or not plus folder to save them to (will be subdir of out_subdir)
    "save_projections": True,
    "projections_subdir": 'vis'
}
    
notebook_to_run = "/home/stumberger/ep2024/subscripts/apply_alignment_to_images.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 4) Align FISH images

In [ ]:
parameters_list = [
    {
        "base_path_target": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/",
        "base_path_moving": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"
    }
]

parameters_common = {
    "alignment_params_file" : '/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/transformations_local_1round.json',
    "exclude_pattern_target" : "sted",
    "include_pattern_target" : None,
    "exclude_pattern_moving" : None,
    "include_pattern_moving" : None,

    "channels_to_include_target" : [0,1],
    "channels_to_include_moving" : [0, ],
    
    # whether to fuse multiple moving images
    # if False, will only transform the one with the highest overlap, ignoring other moving tiles at the border of target image
    "fuse_multiple_moving" : True,
    
    # "out_subdir" : 'aligned_beads_correlation',
    "out_subdir" : 'aligned_confocal',
    
    # whether to save projections or not plus folder to save them to (will be subdir of out_subdir)
    "save_projections": True,
    "projections_subdir": 'vis'
}
    
notebook_to_run = "/home/stumberger/ep2024/subscripts/apply_alignment_to_images.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)